# Busca de dados - Copa\n
\n
Use este notebook para coletar os dados da Copa (16 times) e gerar os arquivos em `datasets_copa`.

In [1]:
import cartolafc
import pandas as pd
import json
import time
import requests
from functools import lru_cache

pd.set_option('display.max_columns', 50)            # permite a visualizacao de 50 colunas do dataframe
pd.options.display.float_format = '{:.2f}'.format   # pandas: para todos os numeros aparecerem com duas casas decimais

# Cria uma instancia da API
api = cartolafc.Api(attempts=5)

# Constantes do 1? turno
INICIO_COPA = 16
FIM_COPA = 19
COLUNAS_RODADAS = [f"Rodada {r}" for r in range(INICIO_COPA, FIM_COPA + 1)]

2026-01-27 15:32:07,037 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


In [2]:
# Lista fixa de IDs dos participantes (lista de int)
ids_participantes = [
    # 19209079, 1488983, 287965, 2916559, 186283, 16411206, 47775950, 1747619, 32966, 44810918,
    # 1867254, 4088673, 184499, 1273719, 5823700, 3708025
]

In [3]:
HEADERS = {
    "User-Agent": "Mozilla/5.0",
    "Accept": "application/json,text/plain,*/*",
    "Referer": "https://cartola.globo.com/",
}

@lru_cache(maxsize=5000)
def nome_time_por_id_api(time_id: int, timeout=15) -> str:
    endpoints = [
        f"https://api.cartolafc.globo.com/time/id/{time_id}",
        f"https://api.cartolafc.globo.com/time/{time_id}",
    ]

    for url in endpoints:
        for tentativa in range(3):
            try:
                r = requests.get(url, headers=HEADERS, timeout=timeout)
                if r.status_code == 429:
                    time.sleep(0.8 + tentativa * 0.8)
                    continue
                if r.status_code != 200:
                    break

                data = r.json()
                if isinstance(data, dict):
                    if isinstance(data.get("time"), dict) and isinstance(data["time"].get("nome"), str):
                        return data["time"]["nome"]
                    if isinstance(data.get("nome"), str):
                        return data["nome"]
                break
            except Exception:
                time.sleep(0.5)
                continue

    return f"Time {time_id}"

In [4]:
# Base com todos os participantes
if not isinstance(ids_participantes, list) or not ids_participantes:
    raise ValueError("ids_participantes precisa ser uma lista de IDs")

ids_participantes = list(dict.fromkeys(ids_participantes))

df_base = pd.DataFrame({"time_id": ids_participantes}).drop_duplicates()
df_base["Time"] = df_base["time_id"].apply(nome_time_por_id_api)

df_base = df_base.set_index("time_id").sort_index()

# Dicionario Nome -> ID (compatibilidade com codigo legado)
ids_times_dict = {row["Time"]: row["time_id"] for _, row in df_base.reset_index().iterrows()}

# Links para o Excel
df_urls = pd.DataFrame({
    "Nome do Time": df_base["Time"].values,
    "ID do Time": df_base.index.values,
})

df_urls["Link do Time"] = df_urls["ID do Time"].apply(
    lambda x: f"https://cartola.globo.com/#!/time/{x}"
)

df_urls = df_urls[["Nome do Time", "ID do Time", "Link do Time"]]

caminho_excel = "links_times_cartola_copa.xlsx"
df_urls.to_excel(caminho_excel, index=False)
print(f"? Arquivo salvo com sucesso: {caminho_excel}")

display(df_base)
# display(df_urls)

ValueError: ids_participantes precisa ser uma lista de IDs

In [ ]:
def campeonato_comecou(ids, rodada_ref=INICIO_COPA):
    lista_ids = list(ids.values()) if isinstance(ids, dict) else list(ids)
    for time_id in lista_ids:
        try:
            t = api.time(time_id=time_id, rodada=rodada_ref)
            v = getattr(t, "ultima_pontuacao", None)
            if v is not None:
                return True
        except Exception:
            continue
    return False

In [ ]:
# GERAR copa_dados.js
# Ajuste a ordem dos confrontos aqui quando o Cartola divulgar.
import json
from pathlib import Path

# Mapa id -> nome (a partir do df_base)
mapa_nomes = df_base.reset_index().set_index('time_id')['Time'].to_dict()

# Times (ordem conforme ids_participantes)
times = [
    {"id": int(time_id), "nome": mapa_nomes.get(int(time_id), f"Time {time_id}"), "turnoPts": None}
    for time_id in ids_participantes
]

# Confrontos das oitavas (ordem manual)
confrontos_oitavas = [
    (19209079, 1747619),
    (3708025, 32966),
    (186283, 4088673),
    (2916559, 184499),
    (1488983, 5823700),
    (47775950, 44810918),
    (16411206, 1867254),
    (287965, 1273719),
]

copa_dados = {
    "temporada": "2026/1",
    "times": times,
    "fases": {
        "oitavas": [
            {"casaId": casa, "foraId": fora, "casaPts": None, "foraPts": None}
            for casa, fora in confrontos_oitavas
        ],
        "quartas": [],
        "semi": [],
        "final": [],
        "terceiro": [],
    },
}

saida = Path('copa_dados.js')
conteudo_js = "window.copaDados = " + json.dumps(copa_dados, ensure_ascii=False, indent=2) + ";"
saida.write_text(conteudo_js, encoding='utf-8')
print(f'Arquivo gerado: {saida.resolve()}')


Arquivo gerado: C:\Users\ferna\Projetos\GitHub\cartola_2026\copa\datasets_copa\copa_dados.js
